# Setup

In [1]:
import os
import json
import lancedb
# import dsutils
import numpy as np
import pyarrow as pa
from glob import glob
from tqdm import tqdm
from docarray.typing import NdArray
from typing import Optional
from docarray import BaseDoc, DocList
from FlagEmbedding import BGEM3FlagModel
from docarray.index import HnswDocumentIndex
from pymongo import MongoClient
from bson import ObjectId


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [2]:
class Document(BaseDoc):
    _id: str
    timestamp: str
    query: str
    title: str
    press: str
    summary: Optional[str] = None
    content: str
    url: str
    origin: str
    embedding: Optional[NdArray] = None

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "summary" in "Document" shadows an attribute in parent "BaseDoc"
  warnings.warn(


## MongoDB

In [3]:
client = MongoClient()
hostname = 'mongo.stockhelper-mongodb.store'
username = 'root'
password = 'financial'
client = MongoClient(hostname, username=username, password=password)
db = client['financial']
news = db.news.find({})

In [4]:
def convert_objectid(doc):
    if "_id" in doc:
        doc["_id"] = str(doc["_id"])
        
    if 'timestamp' in doc:
        doc['timestamp'] = doc['timestamp'].strftime("%Y-%m-%d %H:%M:%S")

    return doc

In [5]:
news = list(news)
news = [convert_objectid(doc) for doc in news]

OperationFailure: Encountered non-retryable error during query :: caused by :: Request 1149301 timed out, deadline was 2024-06-09T08:21:47.336+00:00, op was RemoteCommand 1149301 -- target:[mongodb-mongodb-sharded-configsvr-0.mongodb-mongodb-sharded-headless.mongo.svc.cluster.local:27017] db:config expDate:2024-06-09T08:21:47.336+00:00 cmd:{ find: "shards", maxTimeMS: 30000, readConcern: { level: "snapshot", afterClusterTime: Timestamp(1717832188, 1) } }, full error: {'ok': 0.0, 'errmsg': 'Encountered non-retryable error during query :: caused by :: Request 1149301 timed out, deadline was 2024-06-09T08:21:47.336+00:00, op was RemoteCommand 1149301 -- target:[mongodb-mongodb-sharded-configsvr-0.mongodb-mongodb-sharded-headless.mongo.svc.cluster.local:27017] db:config expDate:2024-06-09T08:21:47.336+00:00 cmd:{ find: "shards", maxTimeMS: 30000, readConcern: { level: "snapshot", afterClusterTime: Timestamp(1717832188, 1) } }', 'code': 202, 'codeName': 'NetworkInterfaceExceededTimeLimit', 

In [23]:
print(news[0])
print(len(news))

{'_id': '665eb4f4098aeb5dfb7f255d', 'timestamp': '2024-05-28 23:30:10', 'query': 'KB금융', 'title': '키움증권, 상장사 첫 밸류업 공시…KB금융 예고공시', 'press': '연합뉴스TV', 'summary': None, 'content': "키움증권이 상장사 가운데 처음으로 밸류업 계획을 공시했습니다.키움증권은 오늘(28일) 오후 '기업가치 제고 계획'을 공시하면서, 3개년 중기 목표로 자기자본이익률 ROE 15% 이상, 주주환원율 30% 이상 등을 달성하겠다는 내용을 발표했습니다.앞서 어제(27일)는 KB금융이 올해 4분기 중 밸류업 제고 계획을 안내하겠다는 예고 공시를 한 바 있습니다.상장사들의 밸류업 공시는 한국거래소가 출범한 밸류업 공시 통합페이지에 노출될 예정입니다.박지운 기자(zwoonie@yna.co.kr)#밸류업 #저PBR #금감원연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23", 'url': 'https://n.news.naver.com/mnews/article/422/0000662169?sid=101', 'origin': 'http://www.yonhapnewstv.co.kr/MYH20240528022800641/?input=1825m'}
11577


## LanceDB

In [6]:
uri = "/workspace/008_PseudoLab/server/vectorstore"
db = lancedb.connect(uri)
# async_db = await lancedb.connect_async(uri)

In [10]:
schema = pa.schema([
    pa.field('id', pa.string()),
    pa.field('timestamp', pa.timestamp('s')),
    pa.field('query', pa.string()),
    pa.field('title', pa.string()),
    pa.field('press', pa.string()),
    pa.field('summary', pa.string()),
    pa.field('content', pa.string()),
    pa.field('url', pa.string()),
    pa.field('origin', pa.string()),
    pa.field("embedding", pa.list_(pa.float32(), list_size=1024))
])

table = db.create_table("test", schema=schema, mode="overwrite")

[2024-06-09T08:25:42Z WARN  lance::dataset] No existing dataset at /workspace/008_PseudoLab/server/vectorstore/test.lance, it will be created


In [11]:
table = db.open_table('test')
# async_tbl = async_db.open_table('financial')

In [12]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu')

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 330260.16it/s]


In [14]:
news

In [13]:
from tqdm.auto import tqdm

docs = []

for data in tqdm(news):
    tmp = 'timestamp: ' + data['timestamp'] + '\n' \
        + 'title: ' + data['title'] + '\n' \
            + 'content: ' + data['content']
    embedding = embedding_function.encode(tmp, return_dense=True, return_sparse=False)
    data['embedding'] = embedding['dense_vecs']
    docs.append(Document(**data))

docs = DocList[Document](docs)
docs = [dict(d) for d in docs]
table.add(docs)

0it [00:00, ?it/s]


IndexError: list index out of range

In [ ]:
async_db = await lancedb.connect_async(uri)

## LanceDB Test

In [4]:
table = db.open_table('financial')
table.count_rows()
# async_tbl.count_rows()

11409

In [5]:
embedding_function = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu')

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 305410.49it/s]
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
query = '6월 1일 삼성전자의 주식과 관련된 뉴스를 알려줘'
query = embedding_function.encode(query)['dense_vecs']
docs = table.search(query).limit(8).to_list()
# docs = async_tbl.search(query).limit(8).to_list()

In [28]:
sorted([(doc['timestamp'].strftime("%Y-%m-%d %H:%M:%S"), doc['title']) for doc in docs], key=lambda x : x[0])

[('2024-05-29 11:51:09', '삼성전자, 창사 이후 첫 파업 소식에 1% 넘게 하락'),
 ('2024-05-29 13:14:49', '삼성전자, 사상 첫 노조 파업 소식에…2% 하락 전환'),
 ('2024-05-29 19:34:36', '삼성전자 노조 첫 ‘파업 선언’…“다음 달 7일 연차 소진”'),
 ('2024-05-31 09:17:11', '[특징주]겹악재에 내림세 걷던 삼성전자, 1%대 반등'),
 ('2024-06-03 09:14:13', '6월 코스피·코스닥, 동반 상승 출발… 삼성전자 나흘 만에 ‘빨간 불’'),
 ('2024-06-03 09:22:13', '[특징주] 금리 인하 기대 재조명에… 삼성전자, 나흘 만에 1%대 상승'),
 ('2024-06-03 09:28:08', '코스피, 6월 첫 거래 상승 출발…삼성전자 2% 급등'),
 ('2024-06-03 09:33:17', '[특징주]금리 인하 기대에 수출 호조까지...삼성전자, 2% 대 반등')]

In [29]:
query = '최근 삼성전자의 신제품에 대해 알려줘'
query = embedding_function.encode(query)['dense_vecs']
docs = table.search(query).limit(8).to_list()

In [30]:
sorted([(doc['timestamp'].strftime("%Y-%m-%d %H:%M:%S"), doc['title']) for doc in docs], key=lambda x : x[0])

[('2024-05-31 15:06:10', "삼성전자 '뉴스룸 재팬' 개설… 日 시장 공략 강화"),
 ('2024-05-31 15:10:11', "삼성전자 '뉴스룸 재팬' 개설…日 시장 공략 드라이브"),
 ('2024-05-31 15:43:34', '삼성전자, 일본 뉴스룸 개설…글로벌 47번째'),
 ('2024-06-02 10:04:59', "삼성전자, '비스포크 스팀' 신제품 출시"),
 ('2024-06-02 10:05:01', "삼성전자, 로봇청소기 '비스포크 스팀' 신제품 출시"),
 ('2024-06-02 10:05:02', "삼성전자, 로봇청소기 '비스포크 스팀' 신제품 출시"),
 ('2024-06-02 10:05:04', "삼성전자, 로봇청소기 '비스포크 스팀' 신제품 출시"),
 ('2024-06-02 10:05:06', "삼성전자, '비스포크 스팀' 신제품 출시")]